# 调试一下EMA问题
专门调EMA问题
#先做个最简单的测试
#把saver有没有参数的情况都测一下
#然后把整个网络的存储也测一下

In [ ]:
其他内容放到
tensorflow_demo/model_save_restore_practice.ipynb

In [1]:
#读取：不定义变量，直接恢复？那样怎么验证？直接获取所有变量？

# 过程：先全读出来，打印一下所有变量看一下。
先说重点：指定var_list，每个变量都要有自己的名字！！！如果给模型改过变量名，之前的model废了！
保持这个原则，读取正常


常见问题：主要就是找不到变量和形状不匹配两个问题

In [4]:
import tensorflow as tf
import fer_config as config
import fer_forward
import fer_backward
import fer_generator
TEST_INTERVAL_SECS = 5
TEST_NUM = 3589

In [5]:
with tf.Graph().as_default() as g:
    x = tf.placeholder(tf.float32, [TEST_NUM, config.img_width, config.img_height, fer_forward.NUM_CHANNELS])
    y_ = tf.placeholder(tf.float32, [None, fer_forward.OUTPUT_NODE])

    y = fer_forward.forward(x, False)

    ema = tf.train.ExponentialMovingAverage(fer_backward.MOVING_AVERAGE_DECAY)
    ema_restore = ema.variables_to_restore()
    print(type(ema_restore),len(ema_restore))
    print(ema_restore)
    
    for ele2 in tf.all_variables():#比trainable多一个i哦global step：tf.trainable_variables():
        print(ele2.name)
    

    saver = tf.train.Saver(var_list = ema_restore)#参数
#     saver = tf.train.Saver()#没参数版本
    #ema的具体变量，应该都在ckpt中吧
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    img_batch, label_batch = fer_generator.get_tfrecord(TEST_NUM, config.tfRecord_test)
    for i in range(1):
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(config.MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
#                 print(ckpt)
                saver.restore(sess, ckpt.model_checkpoint_path)
#                 print(saver)
                for s in saver:
                    print(s)
        
        
        


<class 'dict'> 24
{'fc2_b_1/ExponentialMovingAverage': <tf.Variable 'fc2_b_1:0' shape=(512,) dtype=float32_ref>, 'fc2_b/ExponentialMovingAverage': <tf.Variable 'fc2_b:0' shape=(512,) dtype=float32_ref>, 'fc2_w/ExponentialMovingAverage': <tf.Variable 'fc2_w:0' shape=(512, 512) dtype=float32_ref>, 'conv1_b/ExponentialMovingAverage': <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, 'bn1/gamma/ExponentialMovingAverage': <tf.Variable 'bn1/gamma:0' shape=(64,) dtype=float32_ref>, 'conv2_w/ExponentialMovingAverage': <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, 'bn1/beta/ExponentialMovingAverage': <tf.Variable 'bn1/beta:0' shape=(64,) dtype=float32_ref>, 'fc1_w/ExponentialMovingAverage': <tf.Variable 'fc1_w:0' shape=(4608, 512) dtype=float32_ref>, 'bn2/gamma/ExponentialMovingAverage': <tf.Variable 'bn2/gamma:0' shape=(64,) dtype=float32_ref>, 'bn3/gamma/ExponentialMovingAverage': <tf.Variable 'bn3/gamma:0' shape=(128,) dtype=float32_ref>, 'fc3_w/ExponentialMovingAv

TypeError: 'Saver' object is not iterable

# 不规范的和规范的变量名定义
{'Variable/ExponentialMovingAverage': <tf.Variable 'Variable:0' shape=(64,) dtype=float32_ref>, 'w1/ExponentialMovingAverage': <tf.Variable 'w1:0' shape=(4608, 512) dtype=float32_ref>, 'Variable_1/ExponentialMovingAverage': <tf.Variable 'Variable_1:0' shape=(64,) dtype=float32_ref>, 'conv2_w/ExponentialMovingAverage': <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, 'conv1_w/ExponentialMovingAverage': <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, 'Variable_5/ExponentialMovingAverage': <tf.Variable 'Variable_5:0' shape=(7,) dtype=float32_ref>, 'Variable_3/ExponentialMovingAverage': <tf.Variable 'Variable_3:0' shape=(512,) dtype=float32_ref>, 'w2/ExponentialMovingAverage': <tf.Variable 'w2:0' shape=(512, 512) dtype=float32_ref>, 'conv3_w/ExponentialMovingAverage': <tf.Variable 'conv3_w:0' shape=(5, 5, 64, 128) dtype=float32_ref>, 'Variable_2/ExponentialMovingAverage': <tf.Variable 'Variable_2:0' shape=(128,) dtype=float32_ref>, 'Variable_4/ExponentialMovingAverage': <tf.Variable 'Variable_4:0' shape=(512,) dtype=float32_ref>, 'w3/ExponentialMovingAverage': <tf.Variable 'w3:0' shape=(512, 7) dtype=float32_ref>}

{'fc1_w/ExponentialMovingAverage': <tf.Variable 'fc1_w:0' shape=(4608, 512) dtype=float32_ref>, 'conv1_w/ExponentialMovingAverage': <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, 'conv2_w/ExponentialMovingAverage': <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, 'conv3_w/ExponentialMovingAverage': <tf.Variable 'conv3_w:0' shape=(5, 5, 64, 128) dtype=float32_ref>, 'fc2_b/ExponentialMovingAverage': <tf.Variable 'fc2_b:0' shape=(512,) dtype=float32_ref>, 'fc2_w/ExponentialMovingAverage': <tf.Variable 'fc2_w:0' shape=(512, 512) dtype=float32_ref>, 'fc2_b_1/ExponentialMovingAverage': <tf.Variable 'fc2_b_1:0' shape=(512,) dtype=float32_ref>, 'conv1_b/ExponentialMovingAverage': <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, 'conv3_b/ExponentialMovingAverage': <tf.Variable 'conv3_b:0' shape=(128,) dtype=float32_ref>, 'fc3_b/ExponentialMovingAverage': <tf.Variable 'fc3_b:0' shape=(7,) dtype=float32_ref>, 'fc3_w/ExponentialMovingAverage': <tf.Variable 'fc3_w:0' shape=(512, 7) dtype=float32_ref>, 'conv2_b/ExponentialMovingAverage': <tf.Variable 'conv2_b:0' shape=(64,) dtype=float32_ref>}

In [2]:
with tf.Graph().as_default() as g:
    x = tf.placeholder(tf.float32, [TEST_NUM, config.img_width, config.img_height, fer_forward.NUM_CHANNELS])
    y_ = tf.placeholder(tf.float32, [None, fer_forward.OUTPUT_NODE])

    y = fer_forward.forward(x, False)

    ema = tf.train.ExponentialMovingAverage(fer_backward.MOVING_AVERAGE_DECAY)
    ema_restore = ema.variables_to_restore()
    print(type(ema_restore),len(ema_restore))
    print(ema_restore)
    
    for ele2 in tf.all_variables():#比trainable多一个i哦global step：tf.trainable_variables():
        print(ele2.name)
    

#     saver = tf.train.Saver(var_list = ema_restore)#参数
    saver = tf.train.Saver()#没参数版本
    #ema的具体变量，应该都在ckpt中吧
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    img_batch, label_batch = fer_generator.get_tfrecord(TEST_NUM, config.tfRecord_test)
    for i in range(1):
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(config.MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
#                 print(ckpt)
                saver.restore(sess, ckpt.model_checkpoint_path)
                print(saver)
        
        
        


<class 'dict'> 12
{'conv2_b/ExponentialMovingAverage': <tf.Variable 'conv2_b:0' shape=(64,) dtype=float32_ref>, 'fc1_w/ExponentialMovingAverage': <tf.Variable 'fc1_w:0' shape=(4608, 512) dtype=float32_ref>, 'conv1_w/ExponentialMovingAverage': <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, 'conv2_w/ExponentialMovingAverage': <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, 'conv3_w/ExponentialMovingAverage': <tf.Variable 'conv3_w:0' shape=(5, 5, 64, 128) dtype=float32_ref>, 'fc2_b/ExponentialMovingAverage': <tf.Variable 'fc2_b:0' shape=(512,) dtype=float32_ref>, 'fc2_w/ExponentialMovingAverage': <tf.Variable 'fc2_w:0' shape=(512, 512) dtype=float32_ref>, 'fc2_b_1/ExponentialMovingAverage': <tf.Variable 'fc2_b_1:0' shape=(512,) dtype=float32_ref>, 'conv1_b/ExponentialMovingAverage': <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, 'fc3_b/ExponentialMovingAverage': <tf.Variable 'fc3_b:0' shape=(7,) dtype=float32_ref>, 'conv3_b/ExponentialMovingAv

{'conv2_b/ExponentialMovingAverage': <tf.Variable 'conv2_b:0' shape=(64,) dtype=float32_ref>, 'fc1_w/ExponentialMovingAverage': <tf.Variable 'fc1_w:0' shape=(4608, 512) dtype=float32_ref>, 'conv1_w/ExponentialMovingAverage': <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, 'conv2_w/ExponentialMovingAverage': <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, 'conv3_w/ExponentialMovingAverage': <tf.Variable 'conv3_w:0' shape=(5, 5, 64, 128) dtype=float32_ref>, 'fc2_b/ExponentialMovingAverage': <tf.Variable 'fc2_b:0' shape=(512,) dtype=float32_ref>, 'fc2_w/ExponentialMovingAverage': <tf.Variable 'fc2_w:0' shape=(512, 512) dtype=float32_ref>, 'fc2_b_1/ExponentialMovingAverage': <tf.Variable 'fc2_b_1:0' shape=(512,) dtype=float32_ref>, 'conv1_b/ExponentialMovingAverage': <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, 'fc3_b/ExponentialMovingAverage': <tf.Variable 'fc3_b:0' shape=(7,) dtype=float32_ref>, 'conv3_b/ExponentialMovingAverage': <tf.Variable 'conv3_b:0' shape=(128,) dtype=float32_ref>, 'fc3_w/ExponentialMovingAverage': <tf.Variable 'fc3_w:0' shape=(512, 7) dtype=float32_ref>}

In [57]:
# help(ema.variables_to_restore)

In [58]:
# help(tf.train.Saver)

# 基本操作 tf_demo/save_model_practice.ipynb
tensorflow_demo/save_model_practice.ipynb